## 多层次/多水平逻辑斯蒂回归

##### 黄荣贵（复旦大学社会学系）

In [1]:
library(foreign)

contraception = read.dta("datasets//Contraception.dta")

head(contraception)

woman,district,use,livch,age,urban
1,1,N,3+,18.4400,Y
2,1,N,0,-5.5599,Y
3,1,N,2,1.4400,Y
4,1,N,3+,8.4400,Y
5,1,N,0,-13.5590,Y
6,1,N,0,-11.5600,Y


检查因变量的分布

In [2]:
table(contraception$use)


   N    Y 
1175  759 

## 随机截距模型

In [3]:
library(lme4)

Warning message:
“package ‘lme4’ was built under R version 3.4.4”Loading required package: Matrix
Warning message:
“package ‘Matrix’ was built under R version 3.4.4”

In [4]:
m_random = glmer(use ~ urban + age + livch + (1|district), 
                 data=contraception, family=binomial)

summary(m_random)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: use ~ urban + age + livch + (1 | district)
   Data: contraception

     AIC      BIC   logLik deviance df.resid 
  2427.6   2466.6  -1206.8   2413.6     1927 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.8225 -0.7659 -0.5085  0.9983  2.7200 

Random effects:
 Groups   Name        Variance Std.Dev.
 district (Intercept) 0.2124   0.4608  
Number of obs: 1934, groups:  district, 60

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.689648   0.147334 -11.468  < 2e-16 ***
urbanY       0.732979   0.119387   6.140 8.28e-10 ***
age         -0.026594   0.007879  -3.375 0.000738 ***
livch1       1.109125   0.157852   7.026 2.12e-12 ***
livch2       1.376341   0.174641   7.881 3.25e-15 ***
livch3+      1.345184   0.179411   7.498 6.49e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Corre

### ICC

In [5]:
0.2124 / (0.2124 + pi^2/3)

[1] 0.06064641

### 对固定效应的解释

中位OR（不是平均的OR）

In [6]:
exp(fixef(m_random))

(Intercept)      urbanY         age      livch1      livch2     livch3+ 
  0.1845844   2.0812705   0.9737565   3.0317056   3.9603822   3.8388943

Odds Ratio的80%置信区间：以urban变量为例

In [7]:
exp(0.732979 + sqrt(2 * 0.2124) * qnorm(0.9))

[1] 4.798253

In [8]:
exp(0.732979+ sqrt(2 * 0.2124) * qnorm(0.1))

[1] 0.9027643

在上述计算中，0.732979是urban的系数，.2154977是随机截距的方差，0.9和0.1对应80%置信水平的两个端点。

### 对随机效应的中位比数（median odds ratio）解读

下面的计算中，0.2124是随机截距的方差

In [9]:
exp(abs(sqrt(2 * 0.2124) * qnorm(0.75)))

[1] 1.552102

详细的解释和计算公式可参见： Larsen, Petersen, & Budtz-Joergensen et al. (2000), Interpreting Parameters in the Logistic Regression Model with Random Effects.

## 随机系数模型

在下面的模型中，截距和年龄变量的系数均设为随机，并且截距随机效应与年龄随机效应之间的协方差不等于零（由软件所估计）。

In [10]:
m_random_coef = glmer(use ~ urban + age + livch + (1 + age|district), 
                      data=contraception, family=binomial)

summary(m_random_coef)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: use ~ urban + age + livch + (1 + age | district)
   Data: contraception

     AIC      BIC   logLik deviance df.resid 
  2430.3   2480.4  -1206.1   2412.3     1925 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.7852 -0.7662 -0.5023  0.9968  2.6328 

Random effects:
 Groups   Name        Variance  Std.Dev. Corr
 district (Intercept) 0.2107677 0.45909      
          age         0.0002152 0.01467  0.47
Number of obs: 1934, groups:  district, 60

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.685832   0.148172 -11.378  < 2e-16 ***
urbanY       0.737297   0.119705   6.159 7.31e-10 ***
age         -0.027909   0.008341  -3.346  0.00082 ***
livch1       1.100209   0.159024   6.919 4.56e-12 ***
livch2       1.368708   0.175136   7.815 5.49e-15 ***
livch3+      1.340654   0.180045   7.446 9.61e-14 ***
---
Signi

下面的模型中，截距和年龄变量的系数均设为随机，但是截距随机效应与年龄随机效应之间的协方差等于零。

In [11]:
m_random_coef_ind = glmer(use ~ urban + age + livch + (age-1|district) + (1|district),
                          data=contraception, family=binomial)

summary(m_random_coef_ind)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: use ~ urban + age + livch + (age - 1 | district) + (1 | district)
   Data: contraception

     AIC      BIC   logLik deviance df.resid 
  2429.1   2473.6  -1206.5   2413.1     1926 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.8223 -0.7649 -0.5027  1.0001  2.6978 

Random effects:
 Groups     Name        Variance  Std.Dev.
 district   age         0.0002211 0.01487 
 district.1 (Intercept) 0.2144070 0.46304 
Number of obs: 1934, groups:  district, 60

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.697480   0.148348 -11.443  < 2e-16 ***
urbanY       0.738003   0.120063   6.147 7.91e-10 ***
age         -0.026734   0.008227  -3.250  0.00116 ** 
livch1       1.117605   0.158764   7.039 1.93e-12 ***
livch2       1.381579   0.175262   7.883 3.20e-15 ***
livch3+      1.350785   0.180037   7.503 6.25e-14 ***
-

与上述模型等价的表述方式为(age || district)

In [12]:
summary(glmer(use ~ urban + age + livch + (age + 1 || district), data=contraception, family=binomial))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: use ~ urban + age + livch + (age + 1 || district)
   Data: contraception

     AIC      BIC   logLik deviance df.resid 
  2429.1   2473.6  -1206.5   2413.1     1926 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.8223 -0.7649 -0.5027  1.0000  2.6978 

Random effects:
 Groups     Name        Variance  Std.Dev.
 district   (Intercept) 0.2144236 0.46306 
 district.1 age         0.0002211 0.01487 
Number of obs: 1934, groups:  district, 60

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.697492   0.148348 -11.443  < 2e-16 ***
urbanY       0.737983   0.120063   6.147 7.91e-10 ***
age         -0.026734   0.008227  -3.250  0.00116 ** 
livch1       1.117628   0.158763   7.040 1.93e-12 ***
livch2       1.381571   0.175261   7.883 3.20e-15 ***
livch3+      1.350814   0.180037   7.503 6.24e-14 ***
---
Signif. codes

### 城市变量设置为随机效应

In [13]:
contraception$urbanBin = car::recode(contraception$urban, "'Y'=1;'N'=0", as.factor = F)

下面的模型将截距和urbanBin这两个变量的随机效应设置为相互独立

In [14]:
summary(glmer(use ~ urbanBin + age + livch + (1 + urbanBin || district), data=contraception, family=binomial))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: use ~ urbanBin + age + livch + (1 + urbanBin || district)
   Data: contraception

     AIC      BIC   logLik deviance df.resid 
  2426.3   2470.8  -1205.1   2410.3     1926 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9507 -0.7583 -0.5029  0.9838  2.7579 

Random effects:
 Groups     Name        Variance Std.Dev.
 district   (Intercept) 0.2322   0.4818  
 district.1 urbanBin    0.2475   0.4975  
Number of obs: 1934, groups:  district, 60

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.698012   0.149735 -11.340  < 2e-16 ***
urbanBin     0.715423   0.148722   4.810 1.51e-06 ***
age         -0.026349   0.007946  -3.316 0.000913 ***
livch1       1.121495   0.159540   7.030 2.07e-12 ***
livch2       1.374286   0.176013   7.808 5.82e-15 ***
livch3+      1.353120   0.181149   7.470 8.04e-14 ***
---
Signif. 